# 1. 策略详情

### 1.1 买入条件
1. 多头排列行情；<br><br>
2. 两次调整，第二次调整未跌破第一次低点，当第二次调整后的反弹超过第一次高点时<br><br>
3. 股价行至五日线之上：收益增加109509

### 1.2 卖出条件
1. 当股价形成新的高点，但对应那天的macd值显著小于上一个高点的macd值<br><br>

### 1.3 备注
1. 止损为m%，止盈为n%<br>

# 2. 交易程序

#### 2.1 导入包

In [62]:
import quantmind as qm
from tabulate import tabulate
import backtrader as bt
import pandas as pd
import numpy as np
import datetime
import random
import time

#### 2.2 定义策略

In [83]:
class TechnicalGraphicsStrategy(bt.Strategy):
    params = dict(
        trailamount=0.0,
        trailpercent=0.05,
        stop_loss = -10,         # 止损
        stop_win = 20,           # 止盈
        limit_percent = 1,
        max_hold = 5000,
        )
    def __init__(self):
        self.trade_size = 100000 # 每次买卖的交易量
        self.remarkable = 0.1 # “显著”值
        self.macd_init = 0.123456789
        
        self.ma_5 = dict() # 5日均线
        self.ma_20 = dict() # 20日均线
        self.ma_60 = dict() # 60日均线
        self.macd = dict() # macd线
        self.trend = dict() # 记录股价相对前一天是上涨还是下降
        
        self.low_reversed = dict() # 记录之前是否捕捉到低点
        self.adjust_valid = dict() # 记录是否出现第二次调整
        self.low = dict() # 记录第一次调整低点
        self.high = dict() # 记录第二次调整高点
        self.macd_high = dict() # 记录上一次高点的macd值
        self.orders = dict()
        self.hold = {}
        self.stk = self.datas[1:]
        for data in self.stk:
            self.orders[data] = None
            self.hold[data] = None
            
            self.ma_5[data] = bt.indicators.SMA(data.close, period = 5)
            self.ma_20[data] = bt.indicators.SMA(data.close, period = 20)
            self.ma_60[data] = bt.indicators.SMA(data.close, period = 60)
            self.macd[data] = bt.indicators.MACD(data.close)
            self.trend[data] = bt.indicators.Momentum(data.close, period = 1)
            self.low_reversed[data] = False
            self.adjust_valid[data] = False
            self.high[data] = -1
            self.low[data] = -1
            self.macd_high[data] = 0.123456
            continue
        
    def prenext(self):
        self.next()

    def nextstart(self):
        self.log('Start To Trade.')
        self.next()
             
    def log(self, arg):
        print('{} {}'.format(self.datetime.date(), arg)) 
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        # 检查订单是否已经执行完成
        # 注意:如果账户资金不够，订单会被拒绝
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            pass
#             self.log('Order Canceled/Margin/Rejected')
        for d in self.stk:
            self.orders[d] = None
        
    def next(self):
        curDate = self.data0.datetime.date(0) # 以第一支股票的时间为基准，避免数据长度不一致问题
        for data in self.stk:
            if curDate != data.datetime.date(0):
                continue
            if self.orders[data]:
                continue
            # 买入
            multi_head = self.ma_5[data][0] > self.ma_20[data][0] and self.ma_20[data][0] > self.ma_60[data][0]
            if multi_head:
                if self.adjust_valid[data] and self.high[data] >= 0 and data.close[0] > self.high[data] and data.close[0]<self.ma_5[data][0] :#是否出现第二次调整，记录第二次调整高点，
                    self.orders[data]=self.buy(data=data, size=self.trade_size)
                    self.low_reversed[data] = False # 状态还原
                    self.adjust_valid[data] = False # 状态还原
                    self.high[data] = -1 # 状态还原
                    self.low[data] = -1 # 状态还原
#                     self.log(" ===BUY CREATE===  CODE:[{}]   PRICE:￥{:0>5}   SHARE:{:>4d}   AMOUNT:￥{:>1f}".\
#                              format(data._name,data.close[0], self.trade_size, self.trade_size*data.close[0]))
                    continue
                    
                if self.trend[data][0]*self.trend[data][-1] < 0 and self.trend[data][0] < 0 and self.low_reversed[data]:#捕捉一个高点
                    self.high[data] = data.close[-1]
                    
                if self.trend[data][0]*self.trend[data][-1] < 0 and self.trend[data][0] > 0:#捕捉一个低点
                    if self.low_reversed[data] and data[-1] > self.low[data]:# 之前捕捉到低点 and 本次低点比上次低点高
                        self.adjust_valid[data] = True # 调整有效，准备捕捉突破点
                        
                    if self.low_reversed[data] and data[-1] <= self.low[data]:# 图形不成立，之前记录的低点作废
                        self.low_reversed[data] = False
                        
                    if not self.low_reversed[data]:# 之前未捕捉到低点
                        self.low[data] = data.close[-1]
                        self.low_reversed[data] = True
            # 卖出
            if self.trend[data][0]*self.trend[data][-1] < 0 and self.trend[data][0] < 0 and self.low_reversed[data]:#捕捉一个高点
                if self.macd_high[data] != self.macd_init and self.getposition(data).size > 0 and\
                    (self.macd_high[data] - self.macd[data][-1])/self.macd_high[data] > self.remarkable: # 之前记录过macd高点，且本次高点“显著小于上次”
                    trade_size = max(self.trade_size, self.getposition(data).size)
                    self.orders[data]=self.sell(data=data, size=self.trade_size)
                    self.macd_high[data] = self.macd_init
#                     self.log("===SELL CREATE===  CODE:[{}]   PRICE:￥{:0>5}   SHARE:{:>4d}   AMOUNT:￥{:>1f}".\
#                              format(data._name,data.close[0], self.trade_size, self.trade_size*data.close[0]))
                    continue
                    
                if self.macd_high[data] == self.macd_init: # 之前未记录过macd高点
                    self.macd_high[data] = self.macd[data][-1]
                    continue
                self.macd_high[data] = self.macd_init
            self.sell_stk(data)
            
    def sell_stk(self,stk):
        ##仓位
        stk_size=self.getposition(stk).size
        
        #持仓成本
    #     if stk_size>0:
        if stk_size>0:#self.hold[stk] is not None and 

            cur_date = stk.datetime.datetime()

            cost = stk_size * self.getposition(stk).price
            # 当前市值
            current_value = stk_size * stk.close[0]
            # 持仓利润
            profit = current_value - cost
            profit_percent = (profit / cost) * 100.0

            if profit_percent < self.p.stop_loss and stk_size>0:
                self.log('- 调出(止损): [%s], 盈亏:%.2f元(%.2f%%), 数量：(%d股)' % (stk._name, profit, profit_percent,stk_size))
                self.orders[stk] = self.sell(data=stk,size=stk_size)
#                 self.hold[stk] = None
            if profit_percent > self.p.stop_win and stk_size>0:
                self.log('- 调出(止盈): [%s], 盈亏:%.2f元(%.2f%%), 数量：(%d股)' % (stk._name, profit, profit_percent,stk_size))
                self.orders[stk] = self.sell(data=stk,size=stk_size)
#                 self.hold[stk] = None

#### 2.3 主进程

In [35]:
def prepare_index_data(start_date, end_date):
#     再导入指数的日数据
    index_day = qm.get_bar('000001.SH', start_date=start_date,end_date=end_date, freq='1d')
    index_day['trade_date'] = pd.to_datetime(index_day['trade_date'],format="%Y%m%d")
    dataa = bt.feeds.PandasData(
                dataname=index_day,
                datetime = 'trade_date',
                open=3,
                high=4,
                low=5,
                close=6,
                volume=7,
                openinterest=-1,
                timeframe=bt.TimeFrame.Days,
               )
    cerebro.adddata(dataa, name='index_day')

In [76]:
class TradeAnalyzer(bt.Analyzer):
    """
    ref - bt's unique trade identifier
    ticker - data feed name
    datein - date and time of trade opening
    pricein - price of trade entry                      买入价
    dir - long or short
    dateout - date and time of trade closing
    priceout - price of trade exit                      卖出价
    chng% - exit price to entry price ratio             卖出价相对买入价的盈利百分比
    pnl - money profit/loss per trade                   盈亏
    pnl% - proft/loss in %s to broker's value at the trade closing  盈亏占交易关闭时账户总值的百分比
    cumpnl - cumulative profit/loss                     累计的盈亏
    size - max position size during trade               交易中最大的成交数量
    value - max trade value                             交易中最大的成交总值
    nbars - trade duration in bars                      交易持续的bar数
    pnl/bar - profit/loss per bar                       平均每根bar的盈亏
    mfe% - max favorable excursion                      最大有利变动幅度
    mae% - max adverse excursion                        最大不利变动幅度
    """

    def get_analysis(self):
        return self.trades

    def __init__(self):
        self.trades = []
        self.cumprofit = 0.0

    def notify_trade(self, trade):
        if trade.isclosed:
            brokervalue = self.strategy.broker.getvalue()
            dir = 'sell'
            if trade.history[0].event.size > 0: dir = 'buy'

            pricein = trade.history[len(trade.history) - 1].status.price
            priceout = trade.history[len(trade.history) - 1].event.price
            datein = bt.num2date(trade.history[0].status.dt)
            dateout = bt.num2date(trade.history[len(trade.history) - 1].status.dt)
            if trade.data._timeframe >= bt.TimeFrame.Days:
                datein = datein.date()
                dateout = dateout.date()

            pcntchange = 100 * priceout / pricein - 100
            pnl = trade.history[len(trade.history) - 1].status.pnlcomm
            pnlpcnt = 100 * pnl / brokervalue
            barlen = trade.history[len(trade.history) - 1].status.barlen
            pbar = pnl / (barlen+1)
            self.cumprofit += pnl

            size = value = 0.0
            for record in trade.history:
                if abs(size) < abs(record.status.size):
                    size = record.status.size
                    value = record.status.value

            highest_in_trade = max(trade.data.high.get(ago=0, size=barlen + 1))
            lowest_in_trade = min(trade.data.low.get(ago=0, size=barlen + 1))
            hp = 100 * (highest_in_trade - pricein) / pricein
            lp = 100 * (lowest_in_trade - pricein) / pricein
            if dir == 'buy':
                mfe = hp
                mae = lp
            if dir == 'sell':
                mfe = -lp
                mae = -hp

            self.trades.append({'ref': trade.ref, 'ticker': trade.data._name,
                                '买入日期': datein, '买入价格': pricein, '卖出日期': dateout, '卖出价格': priceout,
                                'chng%': round(pcntchange, 2), 'pnl': pnl, 'pnl%': round(pnlpcnt, 2),
                                'size': size, 'value': value, 'cumpnl': self.cumprofit,
                                'nbars': barlen, 'pnl/bar': round(pbar, 2),
                                'mfe%': round(mfe, 2), 'mae%': round(mae, 2)})

In [84]:
# 1. 定义变量
# 1.1 起止时间 
start_date = '20191201'
end_date = '20211201'

# 1.2 费率
capital = 20000000
rate = 0.00015
# 1.3 投资标的
num_stks = 100
all_stk = qm.get_stock_list()
all_stk['list_date']=pd.to_datetime(list(all_stk['list_date']),format="%Y%m%d")
# 剔除上市时间短的股票
all_stk = all_stk[(all_stk['market']!='科创板') & (all_stk['list_date']<'20180101')]
# 剔除ST
all_stk = list(set(all_stk['code'])-set(all_stk[all_stk['name'].str.contains('ST')]['code']))

targets = random.sample(all_stk,num_stks)
print("our stocks:",targets)

# 2. 初始化
# 2.1 创建一个大脑
cerebro = bt.Cerebro()
prepare_index_data(start_date, end_date)

# 2.2 添加投资标的
for target in targets:
    # 载入标的数据
    df = qm.get_bar(target, start_date=start_date, end_date=end_date, freq='1d').sort_values(by='trade_date', ascending=True)
    # 转换数据格式
    df['trade_date1'] = pd.to_datetime(df['trade_date'],format='%Y%m%d') # [int64] --> [date_time]
    data = bt.feeds.PandasData(
        dataname=df,
        datetime = 'trade_date1',
        open=3,
        high=4,
        low=5,
        close=6,
        volume=7,
        openinterest=-1,
        timeframe=bt.TimeFrame.Days,
        )

    # 将数据喂给大脑
    cerebro.adddata(data, name=target)

# 2.3 添加策略
cerebro.addstrategy(TechnicalGraphicsStrategy)

# 2.4 设置初始资金和交易费率
cerebro.broker.setcash(capital)
cerebro.broker.setcommission(commission=rate)
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='AnnualReturn')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')
cerebro.addanalyzer(bt.analyzers.TimeDrawDown, _name='TimeDrawDown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='Trade')
cerebro.addanalyzer(TradeAnalyzer, _name='trade_list')

# 3. 交易
results = cerebro.run(tradehistory=True)

# 4. 打印结果
result = results[0]
print('Sharpe Ratio:', result.analyzers.SharpeRatio.get_analysis(),
     '\n\nAnnualReturn:', result.analyzers.AnnualReturn.get_analysis(),
     '\n\nTimeDrawDown:', result.analyzers.TimeDrawDown.get_analysis(),
     '\n\nDrawDown:', result.analyzers.DrawDown.get_analysis(),
      '\n\nPnl:', result.analyzers.Trade.get_analysis()['won'],
       '\n\nPnl:', result.analyzers.Trade.get_analysis()['lost']
   )
trade_list = result.analyzers.trade_list.get_analysis()
# print('\n' + tabulate(trade_list, headers="keys") + '\n')
for t in trade_list:
    print(t)
    
print("\n===============================================================")
print('Origin Capital:{:.2f}   Final Capital:{:.2f}   Gain:{:.2f}'\
      .format(capital,cerebro.broker.getvalue(),cerebro.broker.getvalue() - capital))
print("===============================================================")

our stocks: ['601515.SH', '600500.SH', '300549.SZ', '600328.SH', '603855.SH', '600460.SH', '603286.SH', '603881.SH', '002456.SZ', '000565.SZ', '300609.SZ', '000514.SZ', '000408.SZ', '000950.SZ', '002648.SZ', '600663.SH', '000708.SZ', '002478.SZ', '300151.SZ', '002466.SZ', '002479.SZ', '002369.SZ', '600157.SH', '000034.SZ', '002801.SZ', '002798.SZ', '000955.SZ', '002349.SZ', '002705.SZ', '601877.SH', '600496.SH', '002522.SZ', '603030.SH', '002148.SZ', '300293.SZ', '000545.SZ', '300020.SZ', '300242.SZ', '603998.SH', '300235.SZ', '002686.SZ', '000028.SZ', '002124.SZ', '000889.SZ', '601880.SH', '600985.SH', '300326.SZ', '002149.SZ', '601010.SH', '002532.SZ', '300158.SZ', '600561.SH', '603089.SH', '002738.SZ', '600706.SH', '600279.SH', '000567.SZ', '300660.SZ', '600491.SH', '000985.SZ', '600789.SH', '300443.SZ', '002554.SZ', '603727.SH', '002201.SZ', '600303.SH', '600889.SH', '002500.SZ', '002849.SZ', '300169.SZ', '002497.SZ', '600535.SH', '002270.SZ', '000998.SZ', '601360.SH', '300340.SZ',